In [3]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten,MaxPooling3D,AveragePooling3D, concatenate,Input ,SpatialDropout3D,Dropout
import keras
from math import e
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
from keras.models import load_model

In [4]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')
model = load_model('../../March2021/Observation_models/modelCNN.h5')

In [5]:
# Read Data 
#ModelList=['ACCESS-CM2','CanESM5','IITM-ESM','INM-CM4-8','INM-CM5-0',
 #          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','MPI-ESM1-2-LR','MPI-ESM1-2-HR','EC-Earth3']
#varList=['hurs','ta','tas','ua','uas','vas','wap','zg']
ModelList=['ACCESS-CM2']
gridlist=['gn']

for m,g in zip(ModelList,gridlist) :
    print(m)
    folderString='/home/cccr/diptih/dipti/Data/ssp126/'+m+'/processed/'
    #print(folderString)
    #fName=folderString+'Regrid_'+v+'_day_'+m+'_ssp126_r1i1p1f1_gn_20150101-21001231.nc'
    t2mData=xr.open_dataset(folderString+'Regrid_tas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(t2mData)
    rhData=xr.open_dataset(folderString+'Regrid_hurs_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(rhData)
    v10Data=xr.open_dataset(folderString+'Regrid_vas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(v10Data)
    u10Data=xr.open_dataset(folderString+'Regrid_uas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(u10Data)
    tLevData=xr.open_dataset(folderString+'Regrid_ta_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(tLevData)
    zLevData=xr.open_dataset(folderString+'Regrid_zg_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(zLevData)
    wLevData=xr.open_dataset(folderString+'Regrid_wap_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(wLevData)
    uLevData=xr.open_dataset(folderString+'Regrid_ua_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(uLevData)
    # Calculate wind speed and relative humidity inv  ushear
    ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
    ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    #Calculate inv
    inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
    inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    inv_ds.attrs
    inv_ds.attrs['units']='K'
    inv_ds.attrs['long_name']='t2m - t850'

    #u shear calculation
    ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
    ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    ushear_ds.attrs['units']='s-1'
    ushear_ds.attrs['long_name']='(u10 - u850)/z850'
    


    # AO data
    AOData = xr.open_dataset(m+'-AOindex-NDJF-Daily-2015-2100.nc')
    aoTS=AOData.AO
        #datetimeindex = aoTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #aoTS['time'] = datetimeindex
    Darray=np.zeros((AOData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(aoTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
    AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
    # EU data
    EUData = xr.open_dataset(m+'-EUindex-NDJF-Daily-2015-2100.nc')
    EUData.EU
    euTS=EUData.EU
    #datetimeindex = euTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #euTS['time'] = datetimeindex
    Darray=np.zeros((EUData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(euTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
    EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
    # create mask
    oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
    oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
    oro.attrs
    oro.attrs['units']='meter'
    oro.attrs['long_name']='Orography'
    oro.values[oro.values>500.1]=np.NaN
    mask=oro.values/oro.values
    #AO
    AO5D=AOData.AO.rolling(time=5).mean()

    AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

    #EU
    EU5D=EUData.EU.rolling(time=5).mean()

    EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))
    
    t1=AO5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/AO5DnormAccessCm2.joblib') 
    norm = StandardScaler().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    AO5DAll.values=t1.unstack()

    t1=EU5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/EU5DnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    EU5DAll.values=t1.unstack()

    AO5DAll.values=AO5DAll.values*mask
    AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

    EU5DAll.values=EU5DAll.values*mask
    EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)
    
    t2m=t2mData.tas.shift(time=1)
    ws=ws_ds.ws.shift(time=1)
    rh=rh_ds.rh.shift(time=1)
    inv=inv_ds.inv.shift(time=1)
    w=wLevData.wap.sel(plev=70000,method='nearest').shift(time=1)
    ushear=ushear_ds.ushear.shift(time=1)
    
    t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    
    
    t1=t2mTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/t2mnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    t2mTsAll.values=t1.unstack()

    t1=wsTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/wsnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wsTsAll.values=t1.unstack()

    t1=rhTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/rhnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    rhTsAll.values=t1.unstack()

    t1=invTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/invnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    invTsAll.values=t1.unstack()

    t1=ushearTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/ushearnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    ushearTsAll.values=t1.unstack()


    t1=wTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/ACCESS-CM2/wnormAccessCm2.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wTsAll.values=t1.unstack()
    
    t2mTsAll.values=t2mTsAll.values*mask
    wsTsAll.values=wsTsAll.values*mask
    rhTsAll.values=rhTsAll.values*mask
    invTsAll.values=invTsAll.values*mask
    ushearTsAll.values=ushearTsAll.values*mask
    wTsAll.values=wTsAll.values*mask
    
    t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
    wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
    rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
    invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
    ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
    wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)
    
    t2mt=t2mTsAll.values
    t2mt=t2mt[:,:,:,None]
    t2mt.shape


    wst=wsTsAll.values
    wst=wst[:,:,:,None]
    wst.shape

    rht=rhTsAll.values
    rht=rht[:,:,:,None]
    rht.shape


    invt=invTsAll.values
    invt=invt[:,:,:,None]
    invt.shape

    wt=wTsAll.values
    wt=wt[:,:,:,None]
    wt.shape

    usheart=ushearTsAll.values
    usheart=usheart[:,:,:,None]
    usheart.shape

    aot=AO5DAll.values
    aot=aot[:,:,:,None]
    aot.shape

    eut=EU5DAll.values
    eut=eut[:,:,:,None]
    eut.shape
    
    X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
    X.shape
    
    X_reshape = np.einsum('lkija->klija',X)
    X_reshape.shape
    
    yLR=model.predict(X_reshape)
    y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})
    
    dump(y_predLin_ds.yLR,'Modelplots_future/'+m+'_ssp126.joblib')

    

ACCESS-CM2


In [6]:
# Read Data 
#ModelList=['ACCESS-CM2','CanESM5','IITM-ESM','INM-CM4-8','INM-CM5-0',
 #          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','MPI-ESM1-2-LR','MPI-ESM1-2-HR','EC-Earth3']
#varList=['hurs','ta','tas','ua','uas','vas','wap','zg']
ModelList=['IITM-ESM']
gridlist=['gn']
normList=['IitmEsm']

for m,g,n in zip(ModelList,gridlist,normList) :
    print(m)
    folderString='/home/cccr/diptih/dipti/Data/ssp126/'+m+'/processed/'
    #print(folderString)
    #fName=folderString+'Regrid_'+v+'_day_'+m+'_ssp126_r1i1p1f1_gn_20150101-21001231.nc'
    t2mData=xr.open_dataset(folderString+'Regrid_tas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(t2mData)
    rhData=xr.open_dataset(folderString+'Regrid_hurs_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(rhData)
    v10Data=xr.open_dataset(folderString+'Regrid_vas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(v10Data)
    u10Data=xr.open_dataset(folderString+'Regrid_uas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(u10Data)
    tLevData=xr.open_dataset(folderString+'Regrid_ta_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(tLevData)
    zLevData=xr.open_dataset(folderString+'Regrid_zg_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(zLevData)
    wLevData=xr.open_dataset(folderString+'Regrid_wap_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(wLevData)
    uLevData=xr.open_dataset(folderString+'Regrid_ua_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(uLevData)
    # Calculate wind speed and relative humidity inv  ushear
    ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
    ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    #Calculate inv
    inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
    inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    inv_ds.attrs
    inv_ds.attrs['units']='K'
    inv_ds.attrs['long_name']='t2m - t850'

    #u shear calculation
    ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
    ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    ushear_ds.attrs['units']='s-1'
    ushear_ds.attrs['long_name']='(u10 - u850)/z850'
    


    # AO data
    AOData = xr.open_dataset(m+'-AOindex-NDJF-Daily-2015-2100.nc')
    aoTS=AOData.AO
        #datetimeindex = aoTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #aoTS['time'] = datetimeindex
    Darray=np.zeros((AOData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(aoTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
    AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
    # EU data
    EUData = xr.open_dataset(m+'-EUindex-NDJF-Daily-2015-2100.nc')
    EUData.EU
    euTS=EUData.EU
    #datetimeindex = euTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #euTS['time'] = datetimeindex
    Darray=np.zeros((EUData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(euTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
    EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
    # create mask
    oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
    oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
    oro.attrs
    oro.attrs['units']='meter'
    oro.attrs['long_name']='Orography'
    oro.values[oro.values>500.1]=np.NaN
    mask=oro.values/oro.values
    #AO
    AO5D=AOData.AO.rolling(time=5).mean()

    AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

    #EU
    EU5D=EUData.EU.rolling(time=5).mean()

    EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))
    
    t1=AO5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/AO5Dnorm'+n+'.joblib') 
    norm = StandardScaler().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    AO5DAll.values=t1.unstack()

    t1=EU5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/EU5Dnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    EU5DAll.values=t1.unstack()

    AO5DAll.values=AO5DAll.values*mask
    AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

    EU5DAll.values=EU5DAll.values*mask
    EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)
    
    t2m=t2mData.tas.shift(time=1)
    ws=ws_ds.ws.shift(time=1)
    rh=rh_ds.rh.shift(time=1)
    inv=inv_ds.inv.shift(time=1)
    w=wLevData.wap.sel(plev=70000,method='nearest').shift(time=1)
    ushear=ushear_ds.ushear.shift(time=1)
    
    t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    
    
    t1=t2mTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/t2mnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    t2mTsAll.values=t1.unstack()

    t1=wsTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wsnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wsTsAll.values=t1.unstack()

    t1=rhTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/rhnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    rhTsAll.values=t1.unstack()

    t1=invTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/invnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    invTsAll.values=t1.unstack()

    t1=ushearTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/ushearnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    ushearTsAll.values=t1.unstack()


    t1=wTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wTsAll.values=t1.unstack()
    
    t2mTsAll.values=t2mTsAll.values*mask
    wsTsAll.values=wsTsAll.values*mask
    rhTsAll.values=rhTsAll.values*mask
    invTsAll.values=invTsAll.values*mask
    ushearTsAll.values=ushearTsAll.values*mask
    wTsAll.values=wTsAll.values*mask
    
    t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
    wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
    rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
    invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
    ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
    wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)
    
    t2mt=t2mTsAll.values
    t2mt=t2mt[:,:,:,None]
    t2mt.shape


    wst=wsTsAll.values
    wst=wst[:,:,:,None]
    wst.shape

    rht=rhTsAll.values
    rht=rht[:,:,:,None]
    rht.shape


    invt=invTsAll.values
    invt=invt[:,:,:,None]
    invt.shape

    wt=wTsAll.values
    wt=wt[:,:,:,None]
    wt.shape

    usheart=ushearTsAll.values
    usheart=usheart[:,:,:,None]
    usheart.shape

    aot=AO5DAll.values
    aot=aot[:,:,:,None]
    aot.shape

    eut=EU5DAll.values
    eut=eut[:,:,:,None]
    eut.shape
    
    X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
    X.shape
    
    X_reshape = np.einsum('lkija->klija',X)
    X_reshape.shape
    
    yLR=model.predict(X_reshape)
    y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})
    
    dump(y_predLin_ds.yLR,'Modelplots_future/'+m+'_ssp126.joblib')

    

IITM-ESM


In [8]:
# Read Data 
#ModelList=['ACCESS-CM2','CanESM5','IITM-ESM','INM-CM4-8','INM-CM5-0',
 #          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','MPI-ESM1-2-LR','MPI-ESM1-2-HR','EC-Earth3']
#varList=['hurs','ta','tas','ua','uas','vas','wap','zg']
ModelList=['EC-Earth3']
gridlist=['gr']
normList=['EcEarth3']

for m,g,n in zip(ModelList,gridlist,normList) :
    print(m)
    folderString='/home/cccr/diptih/dipti/Data/ssp126/'+m+'/processed/'
    #print(folderString)
    #fName=folderString+'Regrid_'+v+'_day_'+m+'_ssp126_r1i1p1f1_gn_20150101-21001231.nc'
    t2mData=xr.open_dataset(folderString+'Regrid_tas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(t2mData)
    rhData=xr.open_dataset(folderString+'Regrid_hurs_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(rhData)
    v10Data=xr.open_dataset(folderString+'Regrid_vas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(v10Data)
    u10Data=xr.open_dataset(folderString+'Regrid_uas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(u10Data)
    tLevData=xr.open_dataset(folderString+'Regrid_ta_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(tLevData)
    zLevData=xr.open_dataset(folderString+'Regrid_zg_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(zLevData)
    wLevData=xr.open_dataset(folderString+'Regrid_wap_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(wLevData)
    uLevData=xr.open_dataset(folderString+'Regrid_ua_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(uLevData)
    # Calculate wind speed and relative humidity inv  ushear
    ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
    ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    #Calculate inv
    inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
    inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    inv_ds.attrs
    inv_ds.attrs['units']='K'
    inv_ds.attrs['long_name']='t2m - t850'

    #u shear calculation
    ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
    ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    ushear_ds.attrs['units']='s-1'
    ushear_ds.attrs['long_name']='(u10 - u850)/z850'
    


    # AO data
    AOData = xr.open_dataset(m+'-AOindex-NDJF-Daily-2015-2100.nc')
    aoTS=AOData.AO
        #datetimeindex = aoTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #aoTS['time'] = datetimeindex
    Darray=np.zeros((AOData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(aoTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
    AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
    # EU data
    EUData = xr.open_dataset(m+'-EUindex-NDJF-Daily-2015-2100.nc')
    EUData.EU
    euTS=EUData.EU
    #datetimeindex = euTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #euTS['time'] = datetimeindex
    Darray=np.zeros((EUData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(euTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
    EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
    # create mask
    oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
    oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
    oro.attrs
    oro.attrs['units']='meter'
    oro.attrs['long_name']='Orography'
    oro.values[oro.values>500.1]=np.NaN
    mask=oro.values/oro.values
    #AO
    AO5D=AOData.AO.rolling(time=5).mean()

    AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

    #EU
    EU5D=EUData.EU.rolling(time=5).mean()

    EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))
    
    t1=AO5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/AO5Dnorm'+n+'.joblib') 
    norm = StandardScaler().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    AO5DAll.values=t1.unstack()

    t1=EU5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/EU5Dnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    EU5DAll.values=t1.unstack()

    AO5DAll.values=AO5DAll.values*mask
    AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

    EU5DAll.values=EU5DAll.values*mask
    EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)
    
    t2m=t2mData.tas.shift(time=1)
    ws=ws_ds.ws.shift(time=1)
    rh=rh_ds.rh.shift(time=1)
    inv=inv_ds.inv.shift(time=1)
    w=wLevData.wap.sel(plev=70000,method='nearest').shift(time=1)
    ushear=ushear_ds.ushear.shift(time=1)
    
    t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    
    
    t1=t2mTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/t2mnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    t2mTsAll.values=t1.unstack()

    t1=wsTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wsnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wsTsAll.values=t1.unstack()

    t1=rhTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/rhnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    rhTsAll.values=t1.unstack()

    t1=invTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/invnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    invTsAll.values=t1.unstack()

    t1=ushearTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/ushearnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    ushearTsAll.values=t1.unstack()


    t1=wTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wTsAll.values=t1.unstack()
    
    t2mTsAll.values=t2mTsAll.values*mask
    wsTsAll.values=wsTsAll.values*mask
    rhTsAll.values=rhTsAll.values*mask
    invTsAll.values=invTsAll.values*mask
    ushearTsAll.values=ushearTsAll.values*mask
    wTsAll.values=wTsAll.values*mask
    
    t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
    wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
    rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
    invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
    ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
    wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)
    
    t2mt=t2mTsAll.values
    t2mt=t2mt[:,:,:,None]
    t2mt.shape


    wst=wsTsAll.values
    wst=wst[:,:,:,None]
    wst.shape

    rht=rhTsAll.values
    rht=rht[:,:,:,None]
    rht.shape


    invt=invTsAll.values
    invt=invt[:,:,:,None]
    invt.shape

    wt=wTsAll.values
    wt=wt[:,:,:,None]
    wt.shape

    usheart=ushearTsAll.values
    usheart=usheart[:,:,:,None]
    usheart.shape

    aot=AO5DAll.values
    aot=aot[:,:,:,None]
    aot.shape

    eut=EU5DAll.values
    eut=eut[:,:,:,None]
    eut.shape
    
    X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
    X.shape
    
    X_reshape = np.einsum('lkija->klija',X)
    X_reshape.shape
    
    yLR=model.predict(X_reshape)
    y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})
    
    dump(y_predLin_ds.yLR,'Modelplots_future/'+m+'_ssp126.joblib')

    

EC-Earth3


In [9]:
# Read Data 
#ModelList=['ACCESS-CM2','CanESM5','IITM-ESM','INM-CM4-8','INM-CM5-0',
 #          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','MPI-ESM1-2-LR','MPI-ESM1-2-HR','EC-Earth3']
#varList=['hurs','ta','tas','ua','uas','vas','wap','zg']
ModelList=['IPSL-CM6A-LR']
gridlist=['gr']
normList=['IpslCm6aLr']

for m,g,n in zip(ModelList,gridlist,normList) :
    print(m)
    folderString='/home/cccr/diptih/dipti/Data/ssp126/'+m+'/processed/'
    #print(folderString)
    #fName=folderString+'Regrid_'+v+'_day_'+m+'_ssp126_r1i1p1f1_gn_20150101-21001231.nc'
    t2mData=xr.open_dataset(folderString+'Regrid_tas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(t2mData)
    rhData=xr.open_dataset(folderString+'Regrid_hurs_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(rhData)
    v10Data=xr.open_dataset(folderString+'Regrid_vas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(v10Data)
    u10Data=xr.open_dataset(folderString+'Regrid_uas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(u10Data)
    tLevData=xr.open_dataset(folderString+'Regrid_ta_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(tLevData)
    zLevData=xr.open_dataset(folderString+'Regrid_zg_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(zLevData)
    wLevData=xr.open_dataset(folderString+'Regrid_wap_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(wLevData)
    uLevData=xr.open_dataset(folderString+'Regrid_ua_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(uLevData)
    # Calculate wind speed and relative humidity inv  ushear
    ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
    ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    #Calculate inv
    inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
    inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    inv_ds.attrs
    inv_ds.attrs['units']='K'
    inv_ds.attrs['long_name']='t2m - t850'

    #u shear calculation
    ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
    ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    ushear_ds.attrs['units']='s-1'
    ushear_ds.attrs['long_name']='(u10 - u850)/z850'
    


    # AO data
    AOData = xr.open_dataset(m+'-AOindex-NDJF-Daily-2015-2100.nc')
    aoTS=AOData.AO
        #datetimeindex = aoTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #aoTS['time'] = datetimeindex
    Darray=np.zeros((AOData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(aoTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
    AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
    # EU data
    EUData = xr.open_dataset(m+'-EUindex-NDJF-Daily-2015-2100.nc')
    EUData.EU
    euTS=EUData.EU
    #datetimeindex = euTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #euTS['time'] = datetimeindex
    Darray=np.zeros((EUData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(euTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
    EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
    # create mask
    oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
    oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
    oro.attrs
    oro.attrs['units']='meter'
    oro.attrs['long_name']='Orography'
    oro.values[oro.values>500.1]=np.NaN
    mask=oro.values/oro.values
    #AO
    AO5D=AOData.AO.rolling(time=5).mean()

    AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

    #EU
    EU5D=EUData.EU.rolling(time=5).mean()

    EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))
    
    t1=AO5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/AO5Dnorm'+n+'.joblib') 
    norm = StandardScaler().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    AO5DAll.values=t1.unstack()

    t1=EU5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/EU5Dnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    EU5DAll.values=t1.unstack()

    AO5DAll.values=AO5DAll.values*mask
    AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

    EU5DAll.values=EU5DAll.values*mask
    EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)
    
    t2m=t2mData.tas.shift(time=1)
    ws=ws_ds.ws.shift(time=1)
    rh=rh_ds.rh.shift(time=1)
    inv=inv_ds.inv.shift(time=1)
    w=wLevData.wap.sel(plev=70000,method='nearest').shift(time=1)
    ushear=ushear_ds.ushear.shift(time=1)
    
    t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    
    
    t1=t2mTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/t2mnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    t2mTsAll.values=t1.unstack()

    t1=wsTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wsnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wsTsAll.values=t1.unstack()

    t1=rhTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/rhnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    rhTsAll.values=t1.unstack()

    t1=invTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/invnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    invTsAll.values=t1.unstack()

    t1=ushearTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/ushearnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    ushearTsAll.values=t1.unstack()


    t1=wTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wTsAll.values=t1.unstack()
    
    t2mTsAll.values=t2mTsAll.values*mask
    wsTsAll.values=wsTsAll.values*mask
    rhTsAll.values=rhTsAll.values*mask
    invTsAll.values=invTsAll.values*mask
    ushearTsAll.values=ushearTsAll.values*mask
    wTsAll.values=wTsAll.values*mask
    
    t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
    wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
    rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
    invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
    ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
    wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)
    
    t2mt=t2mTsAll.values
    t2mt=t2mt[:,:,:,None]
    t2mt.shape


    wst=wsTsAll.values
    wst=wst[:,:,:,None]
    wst.shape

    rht=rhTsAll.values
    rht=rht[:,:,:,None]
    rht.shape


    invt=invTsAll.values
    invt=invt[:,:,:,None]
    invt.shape

    wt=wTsAll.values
    wt=wt[:,:,:,None]
    wt.shape

    usheart=ushearTsAll.values
    usheart=usheart[:,:,:,None]
    usheart.shape

    aot=AO5DAll.values
    aot=aot[:,:,:,None]
    aot.shape

    eut=EU5DAll.values
    eut=eut[:,:,:,None]
    eut.shape
    
    X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
    X.shape
    
    X_reshape = np.einsum('lkija->klija',X)
    X_reshape.shape
    
    yLR=model.predict(X_reshape)
    y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})
    
    dump(y_predLin_ds.yLR,'Modelplots_future/'+m+'_ssp126.joblib')

    

IPSL-CM6A-LR


In [10]:
# Read Data 
#ModelList=['ACCESS-CM2','CanESM5','IITM-ESM','INM-CM4-8','INM-CM5-0',
 #          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','MPI-ESM1-2-LR','MPI-ESM1-2-HR','EC-Earth3']
#varList=['hurs','ta','tas','ua','uas','vas','wap','zg']
ModelList=['MPI-ESM1-2-LR']
gridlist=['gn']
normList=['MpiEsm12Lr']

for m,g,n in zip(ModelList,gridlist,normList) :
    print(m)
    folderString='/home/cccr/diptih/dipti/Data/ssp126/'+m+'/processed/'
    #print(folderString)
    #fName=folderString+'Regrid_'+v+'_day_'+m+'_ssp126_r1i1p1f1_gn_20150101-21001231.nc'
    t2mData=xr.open_dataset(folderString+'Regrid_tas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(t2mData)
    rhData=xr.open_dataset(folderString+'Regrid_hurs_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(rhData)
    v10Data=xr.open_dataset(folderString+'Regrid_vas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(v10Data)
    u10Data=xr.open_dataset(folderString+'Regrid_uas_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(u10Data)
    tLevData=xr.open_dataset(folderString+'Regrid_ta_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(tLevData)
    zLevData=xr.open_dataset(folderString+'Regrid_zg_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(zLevData)
    wLevData=xr.open_dataset(folderString+'Regrid_wap_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(wLevData)
    uLevData=xr.open_dataset(folderString+'Regrid_ua_day_'+m+'_ssp126_r1i1p1f1_'+g+'_20150101-21001231.nc')
    #print(uLevData)
    # Calculate wind speed and relative humidity inv  ushear
    ws = ((v10Data.vas.values**2)+(u10Data.uas.values**2))**0.5
    ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhData.hurs/100.0)},
                   coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})

    #Calculate inv
    inv=t2mData.tas.values-tLevData.ta.sel(plev=85000,method='nearest').values
    inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    inv_ds.attrs
    inv_ds.attrs['units']='K'
    inv_ds.attrs['long_name']='t2m - t850'

    #u shear calculation
    ushear=(uLevData.ua.sel(plev=85000,method='nearest').values-u10Data.uas.values)/(zLevData.zg.sel(plev=85000,method='nearest').values) 
    ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10Data.time,'latitude': v10Data.latitude,'longitude': v10Data.longitude})
    ushear_ds.attrs['units']='s-1'
    ushear_ds.attrs['long_name']='(u10 - u850)/z850'
    


    # AO data
    AOData = xr.open_dataset(m+'-AOindex-NDJF-Daily-2015-2100.nc')
    aoTS=AOData.AO
        #datetimeindex = aoTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #aoTS['time'] = datetimeindex
    Darray=np.zeros((AOData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(aoTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), aoTS[t].values)
    AOData=xr.Dataset({'AO': (('time','latitude','longitude'), Darray)},
                  coords={'time': aoTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude}) 
    # EU data
    EUData = xr.open_dataset(m+'-EUindex-NDJF-Daily-2015-2100.nc')
    EUData.EU
    euTS=EUData.EU
    #datetimeindex = euTS.indexes['time'].to_datetimeindex()
    #datetimeindex
    #euTS['time'] = datetimeindex
    Darray=np.zeros((EUData.time.shape[0],t2mData.latitude.shape[0], t2mData.longitude.shape[0]))
    for t in range(euTS.time.shape[0]) :
        Darray[t,:,:]=np.full((t2mData.latitude.shape[0], t2mData.longitude.shape[0]), euTS[t].values)
    EUData=xr.Dataset({'EU': (('time','latitude','longitude'), Darray)},
                  coords={'time': euTS.time,'latitude': t2mData.latitude,'longitude': t2mData.longitude})
    # create mask
    oro = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100))
    oro.values = OroData.z.sel(latitude=slice(35,0),longitude=slice(50,100)).values/9.81
    oro.attrs
    oro.attrs['units']='meter'
    oro.attrs['long_name']='Orography'
    oro.values[oro.values>500.1]=np.NaN
    mask=oro.values/oro.values
    #AO
    AO5D=AOData.AO.rolling(time=5).mean()

    AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))

    #EU
    EU5D=EUData.EU.rolling(time=5).mean()

    EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),
                                           longitude=slice(50,100))
    
    t1=AO5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/AO5Dnorm'+n+'.joblib') 
    norm = StandardScaler().fit(t1)
    # transform training data
    t1.values = norm.transform(t1)
    AO5DAll.values=t1.unstack()

    t1=EU5DAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/EU5Dnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    EU5DAll.values=t1.unstack()

    AO5DAll.values=AO5DAll.values*mask
    AO5DAll.values=xr.where(np.isnan(AO5DAll.values),  0.000000000001,AO5DAll.values)

    EU5DAll.values=EU5DAll.values*mask
    EU5DAll.values=xr.where(np.isnan(EU5DAll.values),  0.000000000001,EU5DAll.values)
    
    t2m=t2mData.tas.shift(time=1)
    ws=ws_ds.ws.shift(time=1)
    rh=rh_ds.rh.shift(time=1)
    inv=inv_ds.inv.shift(time=1)
    w=wLevData.wap.sel(plev=70000,method='nearest').shift(time=1)
    ushear=ushear_ds.ushear.shift(time=1)
    
    t2mTsAll=t2m[((t2m.time.dt.month>11) | (t2m.time.dt.month<2)) & (t2m.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wsTsAll=ws[((ws.time.dt.month>11) | (ws.time.dt.month<2)) & (ws.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    rhTsAll=rh[((rh.time.dt.month>11) | (rh.time.dt.month<2)) & (rh.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    invTsAll=inv[((inv.time.dt.month>11) | (inv.time.dt.month<2)) & (inv.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    ushearTsAll=ushear[((ushear.time.dt.month>11) | (ushear.time.dt.month<2)) & (ushear.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    wTsAll=w[((w.time.dt.month>11) | (w.time.dt.month<2)) & (w.time.dt.year<2101)].sel(time=slice('2015-1-1','2100-12-31'),latitude=slice(35,0),longitude=slice(50,100))
    
    
    t1=t2mTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/t2mnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    t2mTsAll.values=t1.unstack()

    t1=wsTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wsnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wsTsAll.values=t1.unstack()

    t1=rhTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/rhnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    rhTsAll.values=t1.unstack()

    t1=invTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/invnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    invTsAll.values=t1.unstack()

    t1=ushearTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/ushearnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    ushearTsAll.values=t1.unstack()


    t1=wTsAll.stack(z=("latitude", "longitude"))
    # fit scaler on training data
    norm   = load('../../March2021/'+m+'/wnorm'+n+'.joblib') 
    # transform training data
    t1.values = norm.transform(t1)
    wTsAll.values=t1.unstack()
    
    t2mTsAll.values=t2mTsAll.values*mask
    wsTsAll.values=wsTsAll.values*mask
    rhTsAll.values=rhTsAll.values*mask
    invTsAll.values=invTsAll.values*mask
    ushearTsAll.values=ushearTsAll.values*mask
    wTsAll.values=wTsAll.values*mask
    
    t2mTsAll.values=xr.where(np.isnan(t2mTsAll.values),  0.000000000001,t2mTsAll.values)
    wsTsAll.values=xr.where(np.isnan(wsTsAll.values),  0.000000000001,wsTsAll.values)
    rhTsAll.values=xr.where(np.isnan(rhTsAll.values),  0.000000000001,rhTsAll.values)
    invTsAll.values=xr.where(np.isnan(invTsAll.values),  0.000000000001,invTsAll.values)
    ushearTsAll.values=xr.where(np.isnan(ushearTsAll.values),  0.000000000001,ushearTsAll.values)
    wTsAll.values=xr.where(np.isnan(wTsAll.values),  0.000000000001,wTsAll.values)
    
    t2mt=t2mTsAll.values
    t2mt=t2mt[:,:,:,None]
    t2mt.shape


    wst=wsTsAll.values
    wst=wst[:,:,:,None]
    wst.shape

    rht=rhTsAll.values
    rht=rht[:,:,:,None]
    rht.shape


    invt=invTsAll.values
    invt=invt[:,:,:,None]
    invt.shape

    wt=wTsAll.values
    wt=wt[:,:,:,None]
    wt.shape

    usheart=ushearTsAll.values
    usheart=usheart[:,:,:,None]
    usheart.shape

    aot=AO5DAll.values
    aot=aot[:,:,:,None]
    aot.shape

    eut=EU5DAll.values
    eut=eut[:,:,:,None]
    eut.shape
    
    X=np.array([t2mt,rht,wst,invt,wt,usheart,aot,eut])
    X.shape
    
    X_reshape = np.einsum('lkija->klija',X)
    X_reshape.shape
    
    yLR=model.predict(X_reshape)
    y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR[:,0])}, coords={'time':t2mTsAll.time.values})
    
    dump(y_predLin_ds.yLR,'Modelplots_future/'+m+'_ssp126.joblib')

    

MPI-ESM1-2-LR
